In [5]:
import sys
from tree_sitter import Language, Parser

sys.path.append('/home/CodeT5Experiments/CodeT5/parser/')
from DFG import DFG_python
from parser_utils import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index,
                   tree_to_token_nodes)

In [6]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

In [8]:
dataset['train'][0].keys()

dict_keys(['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'])

In [13]:
code_example = dataset['train'][0]['code']
code_example

'def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.\n        """\n        self.replace(TextContent, value=text, cls=cls)'

In [10]:
dfg_function={
'python':DFG_python
}

In [11]:
parsers={}        
for lang in dfg_function:
    LANGUAGE = Language('/home/CodeT5Experiments/CodeT5/parser/my-languages2.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]    
    parsers[lang]= parser


In [14]:
# Add AST.
tree = parsers['python'][0].parse(bytes(code_example,'utf8')) 
root_node = tree.root_node

In [16]:
ast = root_node.sexp()
ast

'(module (function_definition name: (identifier) parameters: (parameters (identifier) (identifier) (default_parameter name: (identifier) value: (string))) body: (block (expression_statement (string)) (expression_statement (call function: (attribute object: (identifier) attribute: (identifier)) arguments: (argument_list (identifier) (keyword_argument name: (identifier) value: (identifier)) (keyword_argument name: (identifier) value: (identifier))))))))'

In [20]:
def preorder_traversal(node):
    result = ""
    result += node.type + ' ' #" " + node.text.decode('utf-8')
    
    # Recursively traverse the children in preorder
    for child in node.children:
        result += preorder_traversal(child)
    
    return result

In [19]:
ast_preorder = preorder_traversal(root_node)
ast_preorder

'module def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.\n        """\n        self.replace(TextContent, value=text, cls=cls)\nfunction_definition def settext(self, text, cls=\'current\'):\n        """Set the text for this element.\n\n        Arguments:\n            text (str): The text\n            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.\n        """\n        self.replace(TextContent, value=text, cls=cls)\ndef def\nidentifier settext\nparameters (self, text, cls=\'current\')\n( (\nidentifier self\n, ,\nidentifier text\n, ,\ndefault_parameter 

In [21]:
ast_preorder = preorder_traversal(root_node)
ast_preorder

'module function_definition def identifier parameters ( identifier , identifier , default_parameter identifier = string " " ) : block expression_statement string " " expression_statement call attribute identifier . identifier argument_list ( identifier , keyword_argument identifier = identifier , keyword_argument identifier = identifier ) '